In [2]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

c:\LawAI\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ds = load_dataset("joonhok-exo-ai/korean_law_open_data_precedents")
train_ds = ds["train"]

print(train_ds)

Dataset({
    features: ['판례정보일련번호', '사건명', '사건번호', '선고일자', '선고', '법원명', '사건종류명', '판결유형', '판시사항', '판결요지', '참조조문', '참조판례', '전문'],
    num_rows: 85660
})


In [4]:
df = train_ds.to_pandas()
df.head()

,판례정보일련번호,사건명,사건번호,선고일자,선고,법원명,사건종류명,판결유형,판시사항,판결요지,참조조문,참조판례,전문
0,85830,토지건물매매무효확인,4280민상278,19470323,선고,대법원,민사,판결,회사소유 부동산을 회사대표자 개인이 계약당사자로서 매도하고 다시 회사대표자자격으로써...,회사소유의 부동산을 회사대표자인 개인이 계약당사자로서 매도하고 다시 회사대표자 자격...,None,None,"【원고, 상고인】\n중앙재목주식회사 우대표자 취체역 김박일\n【피고, 피상고인】\n..."
1,85834,토지건물소유권이전등기,4281민상298,19480402,선고,대법원,민사,판결,잔대금 지불후에 이전등기절차를 이행하기로한 약지와 동시이행과의 관계,매매계약에서 잔대금의 지불을 받은 후에 이전등기의무를 행하기로 약정한 경우의 약지는...,None,None,"【원고, 상고인】\n정석용\n【피고, 피상고인】\n송종순\n【원심판결】\n청주지방법..."
2,85835,부동산소유권이전등기,4281민상362,19480407,선고,대법원,민사,선고,가. 소송위임 흠결의 항변과 당해 이원의 인증 나. 계약해제존재의 가항변에 대한 판...,가. 소송위임장이 사문서인 경우에 당사자 일방이 그 소송위임의 흠결을 주장한 때에는...,"민사소송법 제80조 , 제137조, 제395조",None,"【원고, 피상고인】\n안유길\n【피고, 상고인】\n최석희\n【원심판결】\n서울지방법..."
3,85836,토지소유권이전등기,4281민상417,19480412,선고,대법원,민사,판결,출계한 양자와 생부사망에 의한 소송수계의 적부,타인의 양자로 출계한 자는 그 생부의 상속인이라 할 수 없음으로 생부가 사망한 경우...,민사소송법 제208조,None,"【원고, 상고인】\n【피고, 피상고인】\n【원심판결】\n서울고등법원 1948. 9...."
4,85831,토지건물이전등기,4280민상383,19480527,선고,대법원,민사,판결,가. 매매계약합의해제와 합의해제후 매매대금수수 양사실 변존의 주장은 모순아닌가 나....,가. 매매계약합의해제와 합의해제후 매매대금수수 양사실의 변존을 인정함은 사회통념에 ...,None,None,"【원고, 상고인】\n박종식\n【피고, 피상고인】\n윤엽\n【주 문】\n원판결은 파훼..."


In [5]:
df.columns

Index(['판례정보일련번호', '사건명', '사건번호', '선고일자', '선고', '법원명', '사건종류명', '판결유형', '판시사항',
       '판결요지', '참조조문', '참조판례', '전문'],
      dtype='object')

In [6]:
def normalize_date(x):
    if pd.isna(x):
        return None
    x = str(int(x))
    if len(x) == 8:        # YYYYMMDD
        return f"{x[:4]}-{x[4:6]}-{x[6:]}"
    elif len(x) == 4:      # YYYY
        return f"{x}-01-01"
    return None

df["선고일자_norm"] = df["선고일자"].apply(normalize_date)


In [7]:
text_fields = [
    "사건명",
    "판시사항",
    "판결요지",
    "전문"
]

for col in text_fields:
    df[col] = df[col].fillna("")

In [8]:
def build_case_text(row):
    parts = []
    
    if row["사건명"]:
        parts.append(f"[사건명] {row['사건명']}")
    if row["판시사항"]:
        parts.append(f"[판시사항] {row['판시사항']}")
    if row["판결요지"]:
        parts.append(f"[판결요지] {row['판결요지']}")
    if row["전문"]:
        parts.append(f"[전문] {row['전문']}")
        
    return "\n".join(parts)

tqdm.pandas()
df["case_text"] = df.progress_apply(build_case_text, axis=1)


100%|██████████| 85660/85660 [00:02<00:00, 29057.01it/s]


In [9]:
final_cols = [
    "판례정보일련번호",
    "사건번호",
    "사건명",
    "법원명",
    "사건종류명",
    "판결유형",
    "선고일자_norm",
    "참조조문",
    "case_text"
]

df_final = df[final_cols]
df_final.head()


,판례정보일련번호,사건번호,사건명,법원명,사건종류명,판결유형,선고일자_norm,참조조문,case_text
0,85830,4280민상278,토지건물매매무효확인,대법원,민사,판결,1947-03-23,None,[사건명] 토지건물매매무효확인\n[판시사항] 회사소유 부동산을 회사대표자 개인이 계...
1,85834,4281민상298,토지건물소유권이전등기,대법원,민사,판결,1948-04-02,None,[사건명] 토지건물소유권이전등기\n[판시사항] 잔대금 지불후에 이전등기절차를 이행하...
2,85835,4281민상362,부동산소유권이전등기,대법원,민사,선고,1948-04-07,"민사소송법 제80조 , 제137조, 제395조",[사건명] 부동산소유권이전등기\n[판시사항] 가. 소송위임 흠결의 항변과 당해 이원...
3,85836,4281민상417,토지소유권이전등기,대법원,민사,판결,1948-04-12,민사소송법 제208조,[사건명] 토지소유권이전등기\n[판시사항] 출계한 양자와 생부사망에 의한 소송수계의...
4,85831,4280민상383,토지건물이전등기,대법원,민사,판결,1948-05-27,None,[사건명] 토지건물이전등기\n[판시사항] 가. 매매계약합의해제와 합의해제후 매매대금...


In [10]:
df_final["text_length"] = df_final["case_text"].apply(len)

df_final = df_final[
    (df_final["text_length"] > 200) & 
    (df_final["text_length"] < 12000)
]

df_final.reset_index(drop=True, inplace=True)


C:\Users\human-23\AppData\Local\Temp\ipykernel_10008\2203706632.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["text_length"] = df_final["case_text"].apply(len)


In [11]:
df_final.to_parquet("korean_precedents_clean.parquet")
df_final.to_csv("korean_precedents_clean.csv", index=False, encoding="utf-8-sig")
